In [57]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from datetime import datetime, time
from tqdm.notebook import tqdm
tqdm.pandas()

import mplfinance as mpf
from mplfinance.original_flavor import candlestick2_ohlc


# Library configuration
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# Set output options
pd.set_option('display.max_columns', 3000)
pd.options.display.max_rows = 999


TRADING_SYSTEM_DB = 'tradingsystemdb'
TRADING_SYSTEM_DB_USER = 'adammiyauchi'
TRADING_SYSTEM_DB_PORT = '4321'
# TRADING_SYSTEM_DB_HOST = '192.168.1.5'
TRADING_SYSTEM_DB_HOST = 'localhost'

Save minute data for pgco30 to data/minute_data/mover_d1_pgco30_<start_date>_<end_date>.feather

In [3]:

for i in range(2013, 2023):
    conn = psycopg2.connect(dbname=TRADING_SYSTEM_DB, user=TRADING_SYSTEM_DB_USER, port=TRADING_SYSTEM_DB_PORT, host=TRADING_SYSTEM_DB_HOST)
    cur = conn.cursor()

    start_date = f'{i}-01-01'
    end_date = f'{i}-12-31'

    cur.execute("""
        SELECT min.*
        FROM minute_data min, mover_extra_pgco30 m
        WHERE min.date::DATE = m.date AND min.symbol = m.symbol AND m.date >= %s AND m.date <= %s
        ORDER BY min.date::DATE, min.symbol, min.date ASC
    """, (start_date, end_date))
    df_raw = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    df_raw.rename(columns={'date': 'datetime'}, inplace=True)

    if df_raw.shape[0] > 0:
        df_raw.to_feather(f'data/minute_data/mover_d1_pgco30_{start_date}_{end_date}.feather')



Merge all pgco30 minute data into a sigle .feather file

In [6]:
import os 

min_data_mover_d1_pgco30_all = pd.DataFrame()
path = 'data/minute_data/'
for file in os.listdir(path):
    df = pd.read_feather(path + file)

    df['time'] = df['datetime'].dt.time
    df['date'] = df['datetime'].dt.date
    # df = df.set_index('datetime')
    df['close'] = df['close'].astype(np.float64)
    df['high'] = df['high'].astype(np.float64)
    df['low'] = df['low'].astype(np.float64)
    df['open'] = df['open'].astype(np.float64)
    df['vwap'] = df['vwap'].astype(np.float64)

    min_data_mover_d1_pgco30_all = pd.concat([
        min_data_mover_d1_pgco30_all,
        df
    ])

min_data_mover_d1_pgco30_all = min_data_mover_d1_pgco30_all.reset_index(drop=True)
min_data_mover_d1_pgco30_all.to_feather(path + 'mover_d1_pgco30_2013_2022.feather')
del min_data_mover_d1_pgco30_all
del df


Generate charts

In [54]:
def create_charts(df, filename, step_length=1, window_length=30, prediction_length=5):      # filename: name of file to save chart metadata to
    files = []
    price_returns, pct_returns, log_returns = [], [], []
    is_pm, is_reg, is_ah = [], [], []    # Is the feature and prediction window during regular trading hours, PM, or AH?

    def create_charts_one_day(df_one):
        for i in range(df_one.shape[0] - window_length - prediction_length + step_length):
            feature_index = i, i+window_length                                  # [feature window start index, feature window end index)
            label_index = i+window_length, i+window_length+prediction_length    # [prediction window start index, prediction window end index)

            window = df_one.iloc[feature_index[0]:feature_index[1],:]
            label = df_one.iloc[label_index[0]:label_index[1], :]

            pred_open_price = label.loc[label.index[0], 'open']
            pred_close_price = label.loc[label.index[-1], 'close']
            price_return = pred_close_price - pred_open_price
            log_return = np.log(pred_close_price / pred_open_price) * 100
            pct_return = (pred_close_price - pred_open_price) / pred_open_price * 100

            pred_end_time = label.loc[label.index[-1], 'time']
            pm = pred_end_time < time(9, 30)
            reg = pred_end_time >= time(9, 30) and pred_end_time < time(16, 0)
            ah = pred_end_time >= time(16, 0)

            plt.figure(figsize=(1.12, 1.12), dpi=100, num=1, clear=True)
            candlestick2_ohlc(
                plt.subplot(211), 
                window['open'].values, 
                window['high'].values,
                window['low'].values, 
                window['close'].values, 
                width=0.6,
                colorup='green', 
                colordown='red', 
                alpha=1
            )
            plt.axis('off')
            plt.subplot(212)
            plt.bar(range(30), window['volume'].values, color='blue')
            plt.axis('off')
            plt.tight_layout()
            
            date, symbol = str(window.loc[window.index[0], 'date']), window.loc[window.index[0], 'symbol']      
            start_time, end_time = str(window.loc[window.index[0], 'time']).replace(':',''), str(window.loc[window.index[-1], 'time']).replace(':','')
            filename = f'data/charts/{window_length}min_features/{prediction_length}min_prediction/{date}_{symbol}_{start_time}_{end_time}.png'
            plt.savefig(filename)
            plt.close('all')

            files.append(filename)
            pct_returns.append(pct_return)
            log_returns.append(log_return)
            price_returns.append(price_return)
            is_pm.append(pm)
            is_reg.append(reg)
            is_ah.append(ah)

    df.groupby(['symbol', 'date']).apply(lambda df: create_charts_one_day(df))

    res = pd.DataFrame({
        'filepath': files,
        'price_returns': price_returns,
        'log_returns': log_returns,
        'pct_returns': pct_returns,
        'is_pm': is_pm,
        'is_reg': is_reg,
        'is_ah': is_ah
    })
    res.to_feather(f'data/chart_metadata/30min_features/5min_prediction/{filename}.feather')
    return res


In [55]:
df = pd.read_feather('data/minute_data/mover_d1_pgco30_2013_2022.feather')

In [56]:
create_charts(df, 'pgco30_chartmetadata')

,filepath,price_returns,log_returns,pct_returns,is_pm,is_reg,is_ah
0,data/charts/30min_features/5min_prediction/201...,-0.09,-1.441178,-1.430843,True,False,False
1,data/charts/30min_features/5min_prediction/201...,0.00,0.000000,0.000000,True,False,False
2,data/charts/30min_features/5min_prediction/201...,0.11,1.798904,1.815182,True,False,False
3,data/charts/30min_features/5min_prediction/201...,0.17,2.784783,2.823920,True,False,False
4,data/charts/30min_features/5min_prediction/201...,0.11,1.804807,1.821192,True,False,False
...,...,...,...,...,...,...,...
1042862,data/charts/30min_features/5min_prediction/201...,0.10,1.025650,1.030928,False,False,True
1042863,data/charts/30min_features/5min_prediction/201...,0.10,1.025650,1.030928,False,False,True
1042864,data/charts/30min_features/5min_prediction/201...,0.15,1.534557,1.546392,False,False,True
1042865,data/charts/30min_features/5min_prediction/201...,0.19,1.939826,1.958763,False,False,True
